# Generating Music with OpenAI's Jukebox Pre-Trained Model
Jukebox is a state-of-the-art neural network model developed by OpenAI, capable of generating high-quality music in a variety of genres and styles. It stands out due to its ability to generate not only melody but also lyrics and singing in tandem. This makes Jukebox an ideal tool for experimenting with creative music generation.

Using OpenAI's pre-trained Jukebox model, I will fine-tune it to generate music based on input audio from my WaveNet & LSTM models.

# MIDI to WAV Conversion using FluidSynth

I will utilize the `FluidSynth` library to convert MIDI files into WAV format. This will create audio files from MIDI sequences that can be used as input for the OpenAI's Jukebox pre-trained model.


In [ ]:
!apt install -y fluidsynth
!pip install midi2audio

In [ ]:
from midi2audio import FluidSynth

# Initialize FluidSynth with a sound font
fs = FluidSynth('/content/gdrive/MyDrive/Violin_Comp_Data/soundfonts/Dark_Violins.sf2')
fs1 = FluidSynth('/content/gdrive/MyDrive/Violin_Comp_Data/soundfonts/Violin_Langtons.sf2')
fs2 = FluidSynth('/content/gdrive/MyDrive/Violin_Comp_Data/soundfonts/Campbell_Violin.SF2')
fs3 = FluidSynth('/content/gdrive/MyDrive/Violin_Comp_Data/soundfonts/Acro_Violins.sf2')
fs4 = FluidSynth('/content/gdrive/MyDrive/Violin_Comp_Data/soundfonts/ensemble_violin.sf2')


# Convert MIDI to WAV
fs.midi_to_audio('/content/gdrive/MyDrive/generated_midi_files/model3_round7.mid', 'model3_round7_dark.wav')
fs1.midi_to_audio('/content/gdrive/MyDrive/generated_midi_files/model3_round7.mid', 'model3_round7_langtons.wav')
fs2.midi_to_audio('/content/gdrive/MyDrive/generated_midi_files/model3_round7.mid', 'model3_round7_campbell.wav')
fs3.midi_to_audio('/content/gdrive/MyDrive/generated_midi_files/model3_round7.mid', 'model3_round7_acro.wav')
fs4.midi_to_audio('/content/gdrive/MyDrive/generated_midi_files/model3_round7.mid', 'model3_round7_ensemble.wav')


In [ ]:
from midi2audio import FluidSynth

# Initialize FluidSynth with a sound font
fs = FluidSynth('/content/gdrive/MyDrive/Violin_Comp_Data/soundfonts/Acro_Violins.sf2')

# Convert MIDI to WAV
fs.midi_to_audio('/content/gdrive/MyDrive/generated_midi_files/baseline1.mid', 'baseline1_acro.wav')
fs.midi_to_audio('/content/gdrive/MyDrive/generated_midi_files/baseline2.mid', 'baseline2_acro.wav')


In [ ]:
from IPython.display import Audio, display
import os

folder_path = '/content/gdrive/MyDrive/generated_wav_files/'

for filename in os.listdir(folder_path):
    if filename.endswith('.wav'):
        file_path = os.path.join(folder_path, filename)
        display(Audio(file_path))


# Jukebox
The below notebook was created using a copy of the original creator's google colab.

**CREDIT TO ORIGINAL CREATOR:** https://colab.research.google.com/drive/1sJda9v46gNzBc7m59MP5zn63AWc-axCY?usp=sharing#scrollTo=sAdFGF-bqVMY

#(Almost) One-click Jukebox notebook with autosaving.

Join the Jukebox community at https://discord.gg/aEqXFN9amV

Video guide for this notebook: https://vimeo.com/817968335

Speed upsampling supported. Switch to upsample mode will happen automatically if data file is detected within the folder provided.

**How to handle memory problems:** In theory, this notebook is crafted to avoid Out of memory errors, but here's some tricks if you still encounter one:
* Restart runtime: At the top of the notebook, click "Runtime" and then "Restart runtime". Then run everything again. You should do this everytime you start a second run within the same session or after you've interrupted one.
* Decrease sample count: Choose a lower number for 'hps.n_samples'.

In [ ]:
#@title Check which GPU you were assigned by running this cell (optional).
!nvidia-smi -i 0 -e 0
!nvidia-smi -L
your_lyrics = """
"""

In [ ]:
your_lyrics = """
"""

In [ ]:
#@title Select your settings and run this cell to start generating

from google.colab import drive
drive.mount('/content/gdrive')

!pip install --upgrade git+https://github.com/craftmine1000/jukebox-saveopt.git

import jukebox
import torch as t
import librosa
import os
from IPython.display import Audio
from jukebox.make_models import make_vqvae, make_prior, MODELS, make_model
from jukebox.hparams import Hyperparams, setup_hparams
from jukebox.sample import sample_single_window, _sample, \
                           sample_partial_window, upsample, \
                           load_prompts
from jukebox.utils.dist_utils import setup_dist_from_mpi
from jukebox.utils.torch_utils import empty_cache
# MPI Connect. MPI doesn't like being initialized twice, hence the following
try:
    if device is not None:
        pass
except NameError:
    rank, local_rank, device = setup_dist_from_mpi()

model = "5b" #@param ["5b_lyrics", "5b", "1b_lyrics"]
if model == '5b':
  your_lyrics = """
  """

save_and_load_models_from_drive = False

#START GDRIVE MODEL LOADER
if save_and_load_models_from_drive == True:
  import os.path
  !apt install pv
  !mkdir /root/.cache ; mkdir /root/.cache/jukebox ; mkdir /root/.cache/jukebox/models ; mkdir /root/.cache/jukebox/models/1b_lyrics ; mkdir /root/.cache/jukebox/models/5b_lyrics ; mkdir /root/.cache/jukebox/models/5b
  !mkdir /content/gdrive/MyDrive/jukebox
  !mkdir /content/gdrive/MyDrive/jukebox/models
  !mkdir /content/gdrive/MyDrive/jukebox/models/5b
  !mkdir /content/gdrive/MyDrive/jukebox/models/5b_lyrics
  !mkdir /content/gdrive/MyDrive/jukebox/models/1b_lyrics


def load_5b_vqvae():
    if os.path.exists("/root/.cache/jukebox/models/5b/vqvae.pth.tar") == False:
      if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b/vqvae.pth.tar") == False:
        print("5b_vqvae not stored in Google Drive. Downloading for the first time.")
        !wget https://openaipublic.azureedge.net/jukebox/models/5b/vqvae.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b/vqvae.pth.tar
      else:
        print("5b_vqvae stored in Google Drive.")
      print('Copying 5b VQVAE')
      !pv /content/gdrive/MyDrive/jukebox/models/5b/vqvae.pth.tar > /root/.cache/jukebox/models/5b/vqvae.pth.tar

def load_1b_lyrics_level2():
  if os.path.exists("/root/.cache/jukebox/models/1b_lyrics/prior_level_2.pth.tar") == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/1b_lyrics/prior_level_2.pth.tar") == False:
      print("1b_lyrics_level_2 not stored in Google Drive. Downloading for the first time. This will take a few more minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/1b_lyrics/prior_level_2.pth.tar -O /content/gdrive/MyDrive/jukebox/models/1b_lyrics/prior_level_2.pth.tar
    else:
      print("1b_lyrics_level_2 stored in Google Drive.")
    print("Copying 1B_Lyrics Level 2")
    !pv /content/gdrive/MyDrive/jukebox/models/1b_lyrics/prior_level_2.pth.tar > /root/.cache/jukebox/models/1b_lyrics/prior_level_2.pth.tar

def load_5b_lyrics_level2():
  if os.path.exists("/root/.cache/jukebox/models/5b_lyrics/prior_level_2.pth.tar") == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b_lyrics/prior_level_2.pth.tar") == False:
      print("5b_lyrics_level_2 not stored in Google Drive. Downloading for the first time. This will take up to 10-15 minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/5b_lyrics/prior_level_2.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b_lyrics/prior_level_2.pth.tar
    else:
      print("5b_lyrics_level_2 stored in Google Drive.")
    print("Copying 5B_Lyrics Level 2")
    !pv /content/gdrive/MyDrive/jukebox/models/5b_lyrics/prior_level_2.pth.tar > /root/.cache/jukebox/models/5b_lyrics/prior_level_2.pth.tar

def load_5b_level1():
  if os.path.exists('/root/.cache/jukebox/models/5b/prior_level_1.pth.tar') == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b/prior_level_1.pth.tar") == False:
      print("5b_level_1 not stored in Google Drive. Downloading for the first time. This may take a few more minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_1.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b/prior_level_1.pth.tar
    else:
      print("5b_level_1 stored in Google Drive.")
    print("Copying 5B Level 1")
    !pv /content/gdrive/MyDrive/jukebox/models/5b/prior_level_1.pth.tar > /root/.cache/jukebox/models/5b/prior_level_1.pth.tar

def load_5b_level0():
  if os.path.exists('/root/.cache/jukebox/models/5b/prior_level_0.pth.tar') == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b/prior_level_0.pth.tar") == False:
      print("5b_level_0 not stored in Google Drive. Downloading for the first time. This may take a few minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_0.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b/prior_level_0.pth.tar
    else:
      print("5b_level_0 stored in Google Drive.")
    print("Copying 5B Level 0")
    !pv /content/gdrive/MyDrive/jukebox/models/5b/prior_level_0.pth.tar > /root/.cache/jukebox/models/5b/prior_level_0.pth.tar

def load_5b_level2():
  if os.path.exists('/root/.cache/jukebox/models/5b/prior_level_2.pth.tar') == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b/prior_level_2.pth.tar") == False:
      print("5b_level_2 not stored in Google Drive. Downloading for the first time. This will take up to 10-15 minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_2.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b/prior_level_2.pth.tar
    else:
      print("5b_level_2 stored in Google Drive.")
  print("Copying 5B Level 2")
  !pv /content/gdrive/MyDrive/jukebox/models/5b/prior_level_2.pth.tar > /root/.cache/jukebox/models/5b/prior_level_2.pth.tar

if save_and_load_models_from_drive == True:
  if model == '5b_lyrics':
    load_5b_vqvae()
    load_5b_lyrics_level2()
    load_5b_level1()
    load_5b_level0()
  if model == '5b':
    load_5b_vqvae()
    load_5b_level2()
    load_5b_level1()
    load_5b_level0()
  elif model == '1b_lyrics':
    load_5b_vqvae()
    load_1b_lyrics_level2()
    load_5b_level1()
    load_5b_level0()
#END GDRIVE MODEL LOADER

hps = Hyperparams()
hps.sr = 44100
hps.n_samples =  2#@param {type:"integer"}
# Specifies the directory to save the sample in.
# We set this to the Google Drive mount point.
hps.name = '/content/gdrive/MyDrive/jukebox_output_7' #@param {type:"string"}
chunk_size = 64 if model in ('5b', '5b_lyrics') else 128
gpu_info = !nvidia-smi -L
if gpu_info[0].find('Tesla T4') >= 0:
  max_batch_size = 2
  print('Tesla T4 detected, max_batch_size set to 2')
elif gpu_info[0].find('Tesla K80') >= 0:
  max_batch_size = 8
  print('Tesla K80 detected, max_batch_size set to 8')
elif gpu_info[0].find('Tesla P100') >= 0:
  max_batch_size = 3
  print('Tesla P100 detected, max_batch_size set to 3')
elif gpu_info[0].find('Tesla V100') >= 0:
  max_batch_size = 3
  print('Tesla V100 detected, max_batch_size set to 3')
elif gpu_info[0].find('A100') >= 0:
  max_batch_size = 6
  print('Tesla A100 detected, max_batch_size set to 6')
else:
  max_batch_size = 3
  print('Different GPU detected, max_batch_size set to 3.')
hps.levels = 3
speed_upsampling = True #@param {type: "boolean"}
if speed_upsampling == True:
  hps.hop_fraction = [1,1,.125]
else:
  hps.hop_fraction = [.5,.5,.125]

vqvae, *priors = MODELS[model]
vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = 1048576)), device)
top_prior = make_prior(setup_hparams(priors[-1], dict()), vqvae, device)

# The default mode of operation.
# Creates songs based on artist and genre conditioning.
mode = 'primed' #@param ["ancestral", "primed"]
if mode == 'ancestral':
  codes_file=None
  audio_file=None
  prompt_length_in_seconds=None
if mode == 'primed':
  codes_file=None
  # Specify an audio file here.
  audio_file = '/content/gdrive/MyDrive/generated_wav_files/baseline1_acro.wav' #@param {type:"string"}
  # Specify how many seconds of audio to prime on.
  prompt_length_in_seconds=5 #@param {type:"slider", min:1, max:23, step:1}

sample_length_in_seconds = 60 #@param {type:"integer"}
if sample_length_in_seconds < 24:
  sample_length_in_seconds = 24
  print('Chosen sample length too low. Automatically set to 24 seconds.')

if os.path.exists(hps.name):
  # Identify the lowest level generated and continue from there.
  for level in [0, 1, 2]:
    data = f"{hps.name}/level_{level}/data.pth.tar"
    if os.path.isfile(data):
      codes_file = data
      if int(sample_length_in_seconds) > int(librosa.get_duration(filename=f'{hps.name}/level_2/item_0.wav')):
        mode = 'continue'
      else:
        mode = 'upsample'
      break

print('mode is now '+mode)
if mode == 'continue':
  print('Continuing from level 2')
if mode == 'upsample':
  print('Upsampling from level '+str(level))

sample_hps = Hyperparams(dict(mode=mode, codes_file=codes_file, audio_file=audio_file, prompt_length_in_seconds=prompt_length_in_seconds))

if mode == 'upsample':
  sample_length_in_seconds=int(librosa.get_duration(filename=f'{hps.name}/level_{level}/item_0.wav'))
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cpu() for z in data['zs']]
  hps.n_samples = zs[-1].shape[0]

if mode == 'continue':
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cpu() for z in data['zs']]
  hps.n_samples = zs[-1].shape[0]

hps.sample_length = (int(sample_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
assert hps.sample_length >= top_prior.n_ctx*top_prior.raw_to_tokens, f'Please choose a larger sampling rate'

# Note: Metas can contain different prompts per sample.
# By default, all samples use the same prompt.

select_artist = "unknown" #@param  ['unknown','andr_s_schiff','sonny_terry','nelly','markus_schulz','modest_petrovich_mussorgsky','otis_redding','aerosmith','kenny_g','james_taylor','bobby_bland','burning_spear','skip_james','heart','tammy_wynette','muse','beres_hammond','james_newton_howard','nelson_freire','benny_goodman','hank_williams','they_might_be_giants','the_brian_jonestown_massacre','lady_gaga','chris_young','alison_krauss_union_station','seal','the_hollies','shabba_ranks','paul_young','iration','buck_owens','the_weeknd','elton_john','smokey_robinson','roy_orbison','headhunterz','blondie','the_temptations','ray_stevens','foo_fighters','christoph_eschenbach','blind_willie_mctell','al_martino','edwin_fischer','victor_young','justin_bieber','styx','doris_day','tex_beneke','the_monkees','richard_wagner','bryan_adams','alessandro_scarlatti','rebelution','pitbull','nat_king_cole','wiz_khalifa','roger_miller','andy_williams','peggy_lee','pyotr_ilyich_tchaikovsky','booker_t_the_mg_s','cilla_black','billy_fury','vera_lynn','enrico_caruso','sly_and_robbie','the_pretenders','the_sweet','kylie_minogue','kay_kyser','san_francisco_symphony','prince','queen','kool_the_gang','horace_andy','midnite','gentleman','wilhelm_kempff','busta_rhymes','the_pogues','def_leppard','al_jolson','king_tubby','hot_chocolate','delroy_wilson','jody_watley','bobby_vee','johnny_mathis','the_rascals','sviatoslav_richter','fred_astaire','john_holt','amy_grant','b_b_king','paul_weston','four_tops','jay_sean','pat_boone','george_frideric_handel','bing_crosby','shalamar','tommy_dorsey','ludacris','kenny_chesney','murray_perahia','lightnin_hopkins','ricky_nelson','clint_mansell','tom_petty_and_the_heartbreakers','gary_glitter','ringo_starr','phil_collins','leo_reisman','al_green','jim_reeves','chris_brown','cliff_edwards','buddy_guy','angelo_badalamenti','frank_sinatra','santana','the_pussycat_dolls','peaches_herb','radu_lupu','the_whispers','eddy_howard','memphis_slim','henry_mancini','giuseppe_verdi','the_dream','vladimir_sofronitsky','u_roy','ken_boothe','the_kinks','howard_shore','hardwell','lou_reed','calvin_harris','eddy_chen','anne_murray','juice_newton','bee_gees','wilson_pickett','alan_jackson','shirley_bassey','waylon_jennings','destiny_s_child','cab_calloway','johnny_copeland','bright_eyes','trey_songz','neil_sedaka','justin_timberlake','arthur_grumiaux','the_who','the_yardbirds','big_joe_turner','duke_ellington','herb_alpert','laura_branigan','michael_jackson','john_denver','peter_gordon','solomon_cutner','steve_miller_band','don_williams','the_pointer_sisters','metallica','the_ink_spots','kenny_rogers','the_game','gene_krupa','snoop_dogg','j_cole','taylor_dayne','r3hab','guy_lombardo_and_his_royal_canadians','shakin_stevens','tim_mcgraw','olivia_newton_john_john_travolta','the_replacements','beenie_man','diplo','sammy_kaye','don_gibson','shaggy','nina_simone','stone_temple_pilots','ne_yo','huddie_william_ledbetter','jerry_goldsmith','the_bellamy_brothers','winifred_atwell','georg_philipp_telemann','timbaland','the_5th_dimension','dionne_warwick','r_kelly','enrique_iglesias','sting','mikey_dread','yellowman','eddie_kendricks','blur','twista','paul_anka','chris_cornell','a_ha','pet_shop_boys','yo_yo_ma','tom_jones','neil_diamond','vic_damone','paul_oakenfold','jascha_heifetz','t_i_','dinah_washington','vladimir_ashkenazy','leif_ove_andsnes','johnny_cash','basement_jaxx','sonic_youth','the_isley_brothers','jason_aldean','henryk_szeryng','lloyd_price','reba_mcentire','bon_jovi','bed_ich_smetana','donny_osmond','chuck_berry','the_smashing_pumpkins','israel_vibration','stan_kenton','conway_twitty_loretta_lynn','mcfly','r_e_m','morgan_heritage','lil_wayne','garnett_silk','lee_scratch_perry','howlin_wolf','jon_secada','goo_goo_dolls','brian_hyland','abc','clarence_gatemouth_brown','wilson_phillips','atlantic_starr','david_guetta','s_rgio_mendes','fr_d_ric_chopin','robert_palmer','charlie_musselwhite','level_42','peter_andre','the_spinners','erasure','philippe_entremont','leo_jan_ek','the_kingston_trio','ronnie_milsap','pat_benatar','robert_casadesus','t_bone_walker','duran_duran','jerry_reed','franz_liszt','journey','steve_angello','showaddywaddy','tears_for_fears','john_williams','daniel_m_ller_schott','tampa_red','tina_turner','the_beatles','miranda_lambert','howard_jones','reo_speedwagon','lady_antebellum','no_doubt','status_quo','tiffany','billy_eckstine','danny_elfman','jimmy_wakely','ike_tina_turner','george_gershwin','dinah_shore','armin_van_buuren','keith_sweat','beaux_arts_trio','arcangelo_corelli','air_supply','w_w','the_mighty_diamonds','harry_gregson_williams','blind_lemon_jefferson','brook_benton','sizzla','bobby_darin','steely_dan','the_skatalites','dwight_yoakam','clara_haskil','jah_cure','cliff_richard','fabolous','sonny_boy_williamson_ii','tinie_tempah','claude_debussy','jewel','bob_seger','otis_rush','mikhail_pletnev','gene_autry','blind_blake','ethel_waters','jackie_wilson','big_mama_thornton','mary_wells','the_mills_brothers','rage_against_the_machine','barbra_streisand','frank_crumit','the_clash','dion','all_4_one','t_i','orchestral_manoeuvres_in_the_dark','culture','josh_turner','one_direction','the_rolling_stones','half_pint','the_searchers','vangelis','jackson_browne','the_beach_boys','train','prince_buster','tavares','eve','bessie_smith','trace_adkins','bay_city_rollers','david_allan_coe','rascal_flatts','petula_clark','10cc','50_cent','the_oak_ridge_boys','barry_manilow','truls_m_rk','morrissey','gerry_the_pacemakers','kay_starr','alborosie','engelbert_humperdinck','new_order','tony_bennett','stereophonics','jimmy_reed','akon','echo_the_bunnymen','jamiroquai','stevie_ray_vaughan','ben_e_king','cheap_trick','dusty_springfield','mel_tillis','damian_marley','ruth_etting','westlife','diana_ross','the_shirelles','frankie_laine','sarah_vaughan','ray_price','gordon_lightfoot','eddie_cantor','the_byrds','gary_numan','bonnie_tyler','aswad','brownie_mcghee','joshua_bell','manic_street_preachers','mr_vegas','tanya_tucker','marvin_gaye_tammi_terrell','the_staple_singers','ry_cooder','john_mayall','martina_mcbride','anner_bylsma','magic_sam','avril_lavigne','robert_nighthawk','the_coasters','ace_of_base','joseph_szigeti','artie_shaw','big_bill_broonzy','the_ventures','rick_astley','les_paul','leonid_kogan','hall_oates','three_dog_night','charley_pride','paul_mccartney','alfred_cortot','crystal_gayle','taj_mahal','mary_j_blige','leann_rimes','mildred_bailey','ll_cool_j','lobo','blake_shelton','matt_haimovitz','beastie_boys','johannes_brahms','martha_and_the_vandellas','lou_rawls','the_righteous_brothers','rosalyn_tureck','vince_gill','gary_moore','bad_company','marty_robbins','russ_morgan','david_cassidy','dj_khaled','leonard_pennario','glenn_miller','kings_of_leon','afrojack','cyndi_lauper','trisha_yearwood','diamond_rio','patty_loveless','sugar_minott','willie_nelson','thomas_newman','georgia_gibbs','eric_carmen','ricky_skaggs','earth_wind_fire','peter_tosh','joe_bonamassa','lonnie_johnson','missy_elliott','nicki_minaj','luther_allison','grigory_sokolov','luke_bryan','alanis_morissette','tracy_lawrence','sonny_james','brandy','michael_bolton','boswell_sisters','antonio_vivaldi','fritz_kreisler','elvis_costello_the_attractions','woody_herman','korn','hans_zimmer','the_osmonds','electric_light_orchestra','t_rex','van_cliburn','harry_james','glee_cast','simple_minds','abba','the_jam','tanya_stephens','the_statler_brothers','craig_david','moby','xxxtentacion','paul_simon','magic_slim','natasha_bedingfield','ennio_morricone','carpenters','joe_tex','cocoa_tea','the_glee_cast','martha_argerich','charlie_rich','memphis_minnie','sheryl_crow','jan_dean','nick_cave_and_the_bad_seeds','del_shannon','lee_greenwood','gary_lewis_the_playboys','marcelle_meyer','count_basie','harold_melvin_the_blue_notes','the_fray','the_prodigy','alicia_keys','conway_twitty','barrington_levy','b_o_b','tritonal','mario_lanza','mac_davis','billy_murray','igor_stravinsky','pretenders','jordin_sparks','alice_in_chains','ohio_players','rick_springfield','jimmie_rodgers','buju_banton','linda_ronstadt','george_benson','2pac','soulja_boy','the_flaming_lips','ignacy_jan_paderewski','gloria_estefan','ariana_grande','black_uhuru','tony_pastor','sublime','snow_patrol','daft_punk','johnny_winter','robbie_williams','eddie_rabbitt','james_cotton','brad_paisley','manfred_mann','bassnectar','margaret_whiting','sam_cooke','robert_cray','the_beautiful_south','barbara_mandrell','dick_haymes','creedence_clearwater_revival','chic','ray_charles','carter_family','ti_sto','survivor','c_line_dion','sergei_prokofiev','b_la_bart_k','the_congos','yefim_bronfman','laidback_luke','darius_rucker','ray_anthony','incubus','carole_king','james_brown','swv','bruno_mars','aphex_twin','nitty_gritty_dirt_band','gustav_mahler','the_shadows','the_moody_blues','reverend_gary_davis','sia','gaetano_donizetti','earl_hooker','the_commodores','maria_jo_o_pires','eric_donaldson','elmore_james','sean_kingston','don_carlos','linkin_park','jay_the_americans','grand_funk_railroad','jls','frankie_valli','lefty_frizzell','en_vogue','the_cure','perry_como','johnny_mercer','stevie_wonder','ernest_tubb','ramin_djawadi','ashanti','rosemary_clooney','anne_sophie_mutter','helen_forrest','augustus_pablo','the_carpenters','claudio_arrau','bob_dylan','joe_simon','culture_club','the_ipana_troubadors','jennifer_lopez','karyn_white','joe','sarah_mclachlan','j_geils_band','dean_martin','hank_snow','clyde_mcphatter','tlc','beck','jimmy_dean','roy_acuff','outkast','freddie_mcgregor','gus_arnheim','gramatik','merle_haggard','steve_lawrence','ma_rainey','jimmy_dorsey','johnny_paycheck','arthur_rubinstein','talking_heads','capleton','les_brown','leonard_bernstein','bobby_goldsboro','kendrick_lamar','lenny_kravitz','nat_shilkret','toby_keith','junior_wells','billy_j_kramer_the_dakotas','peter_paul_mary','armand_van_helden','h_sker_d_','alabama','eminem','felix_mendelssohn_bartholdy','richard_goode','pink_floyd','sara_evans','lonnie_donegan','boney_m','deadmau5','lee_ann_womack','eric_clapton','ray_parker_jr','etta_james','the_white_stripes','gary_u_s_bonds','glen_gray_and_the_casa_loma_orchestra','bonnie_raitt','soulja_boy_tell_em','bobby_rydell','carly_simon','koko_taylor','gregory_isaacs','red_hot_chili_peppers','josef_suk','clint_black','buddy_clark','tool','pierre_fournier','alice_cooper','lucky_dube','jorge_bolet','don_diablo','r_l_burnside','klaus_badelt','dolly_parton','james_horner','green_day','henry_burr','the_doors','roxette','louis_armstrong','jerry_butler','louis_prima','paul_van_dyk','dennis_brown','toni_braxton','jerry_lee_lewis','donna_summer','percy_faith','willie_dixon','elvis_costello','youri_egorov','dmitri_shostakovich','webb_pierce','monica','pierre_laurent_aimard','muddy_waters','garth_brooks','boyz_ii_men','kris_kristofferson','duane_eddy','coolio','gidon_kremer','eurythmics','randy_travis','collie_buddz','faith_evans','matisyahu','brooks_dunn','lulu','fletcher_henderson','eek_a_mouse','shlomo_mintz','ray_noble','bill_withers','ub40','b_j_thomas','mariah_carey','olivia_newton_john','jackson_5','michael_rose','three_days_grace','glen_campbell','keith_urban','onerepublic','ted_weems','johnny_desmond','billy_preston','billy_vaughn','otis_spann','marion_harris','richard_marx','frankie_carle','xavier_cugat','zz_top','faz_l_say','new_york_philharmonic','teresa_brewer','pixies','david_oistrakh','rory_gallagher','ted_lewis_his_band','patsy_cline','tracy_byrd','daniel_shafran','johnny_horton','billy_ocean','don_mclean','ginuwine','kiss','kaskade','the_hold_steady','rod_stewart','rudolf_serkin','the_wanted','herman_s_hermits','rusty_draper','jay_z','mississippi_john_hurt','linval_thompson','billie_holiday','lawrence_welk','john_lee_hooker','the_offspring','ramones','ronan_keating','sir_clifford_michael_curzon','dierks_bentley','bush','texas','gene_austin','after_all','depeche_mode','eddy_arnold','ferry_corsten','inxs','jimmie_lunceford','johann_sebastian_bach','maxi_priest','travis_tritt','girls_aloud','meat_loaf','brenda_lee','modest_mouse','glenn_gould','mc_hammer','erik_satie','the_andrews_sisters','alicia_de_larrocha','america','charles_harrison','georges_cziffra','peter_serkin','gladys_knight_the_pips','anton_n_dvo_k','emil_gilels','myra_hess','portugal_the_man','rufus','sergei_rachmaninoff','andr_previn','natalie_cole','the_killers','helen_reddy','billy_idol','shawn_mendes','john_lennon','joan_jett','jimmy_cliff','taylor_swift','shura_cherkassky','marvin_gaye','the_drifters','paul_revere_the_raiders','donovan','alma_gluck','fleetwood_mac','jo_stafford','samson_fran_ois','vaughn_monroe','tennessee_ernie_ford','whitesnake','iron_maiden','blind_boy_fuller','eric_church','atomic_kitten','harry_nilsson','paul_specht','bob_marley','les_baxter','aretha_franklin','jessie_j','robert_johnson','maroon_5','sheena_easton','george_strait','bruce_springsteen','oasis','jack_white','fatboy_slim','the_jesus_and_mary_chain','hank_williams_jr','bill_haley_his_comets','charlie_daniels','pearl_jam','the_stylistics','sean_paul','david_essex','zino_francescatti','bruce_hornsby','system_of_a_down','scott_joplin','benny_benassi','mitch_miller','gloria_gaynor','steve_winwood','drake','bone_thugs_n_harmony','nickelback','john_mellencamp','gene_pitney','chet_atkins','mark_ronson','kc_the_sunshine_band','chingy','maurice_ravel','infected_mushroom','walter_gieseking','pavement','deniece_williams','tommy_james_the_shondells','harry_belafonte','robert_schuman','george_jones','whitney_houston','the_ames_brothers','max_romeo','patti_page','p_nk','vienna_philharmonic','violent_femmes','sly_the_family_stone','johnny_marvin','sammy_davis_jr_','nirvana','the_turtles','the_manhattans','the_supremes','gregg_allman','alton_ellis','giacomo_puccini','artur_schnabel','2_unlimited','michael_rabin','toto','lil_kim','shinedown','teddy_wilson','alpha_blondy','the_guess_who','my_chemical_romance','matchbox_twenty','wolfgang_gartner','ed_sheeran','faron_young','fats_waller','kate_smith','loretta_lynn','antonio_meneses','tony_martin','frankie_vaughan','the_mamas_the_papas','david_bowie','erskine_hawkins','the_four_lads','queens_of_the_stone_age','the_human_league','mud','bread','dixie_chicks','backstreet_boys','connie_francis','the_black_keys','paul_weller','pieter_wispelwey','the_bachelors','janet_jackson','the_association','edwin_starr','hank_williams_jr_','steven_isserlis','the_doobie_brothers','jimi_hendrix','sousa_s_band','ricky_martin','the_cranberries','paul_whiteman','the_saturdays','keb_mo','the_psychedelic_furs','boyzone','the_chemical_brothers','johnny_tillotson','joe_cocker','babyface','wet_wet_wet','tom_petty','lesley_gore','deorro','war','wolfgang_schneiderhan','isaac_hayes','beyonc','gordon_jenkins','flo_rida','captain_tennille','adolf_busch','bobby_bare','johnnie_taylor','above_beyond','robin_thicke','ja_rule','barry_white','florence_the_machine','the_jimi_hendrix_experience','gabrielle','leon_fleisher','yehudi_menuhin','george_harrison','kate_bush','u2','nancy_sinatra','the_everly_brothers','peter_green','cat_stevens','the_new_seekers','itzhak_perlman','red_nichols_his_five_pennies','the_script','ijahman_levi','rihanna','joan_jett_and_the_blackhearts','the_dorsey_brothers_orchestra','charlie_barnet','lmfao','112','kelis','john_mccormack','guns_n_roses','gordon_macrae','buddy_holly','franz_schubert','tommy_roe','garbage','bow_wow','belle_and_sebastian','paul_tortelier','johnny_nash','lazar_berman','avicii','5_seconds_of_summer','mississippi_fred_mcdowell','ivan_moravec','little_richard','john_ogdon','dr_dre','andy_russell','sex_pistols','puddle_of_mudd','june_carter_cash','freddie_king','dino_ciani','h_sker_d','nicky_romero','shania_twain','major_lazer','usher','madonna','nsync','the_o_jays','faith_hill','terence_trent_d_arby','gene_chandler','lisa_stansfield','pharrell_williams','david_geringas','dr_alimantado','jack_scott','vladimir_horowitz','genesis','jessica_simpson','peter_gabriel','spike_jones_and_his_city_slickers','leonard_rose','richard_strauss','mgmt','blink_182','ella_fitzgerald','carrie_underwood','evgeny_kissin','olly_murs','ky_mani_marley','rose_royce','desmond_dekker','montell_jordan','little_river_band','t_pain','gary_allan','chubby_checker','the_diamonds','creed','louis_jordan','vernon_dalhart','gilbert_o_sullivan','ziggy_marley','irene_cara','katy_perry','all_saints','benjamin_britten','p_m_dawn','iona_brown','the_lettermen','natalie_imbruglia','viktoria_mullova','zac_brown_band','bj_rk','mutabaruka','jermaine_jackson','the_impressions','spandau_ballet','luther_vandross','thompson_twins','travis_scott','roots_radics','madness','skrillex','the_smiths','van_halen','the_four_seasons','j_b_lenoir','johnny_rivers','bob_marley_the_wailers','albert_collins','arctic_monkeys','billy_joel','joe_nichols','chicago','coldplay','blue','papa_roach','r_e_m_','eddie_money','ellie_goulding','gym_class_heroes','the_charlie_daniels_band','bette_midler','hubert_sumlin','phil_harris','arturo_benedetti_michelangeli','steps','simon_and_garfunkel','gyptian','britney_spears','lang_lang','blackstreet','the_dave_clark_five','the_chi_lites','wolfgang_amadeus_mozart','johnnie_ray','elvis_presley','red_norvo','the_velvet_underground','nelly_furtado','connee_boswell','soundgarden','dr_hook_the_medicine_show','led_zeppelin','boney_m_','gregor_piatigorsky','john_anderson','joni_james','the_all_american_rejects','marcia_griffiths','ben_bernie','eric_prydz','the_three_suns','albert_king','jodeci','inner_circle','don_cornell','jeff_healey','brian_mcknight','the_stranglers','adam_faith','francis_poulenc','fall_out_boy','the_jets','groundation','randy_newman','jane_s_addiction','bobby_vinton','guiomar_novaes','bo_diddley','philadelphia_orchestra','garrick_ohlsson','evanescence','wilhelm_backhaus','the_platters','the_seekers','selena_gomez','eddy_grant','jan_garber','gioacchino_rossini','christina_aguilera','stevie_nicks','color_me_badd','sandie_shaw','migos','the_allman_brothers_band','will_young','new_edition','radiohead','orbital','the_judds','dan_fogelberg','pascal_rog_','leona_lewis','charlie_patton','yann_tiersen','the_abyssinians','frank_ifield','john_mayer','roger_wolfe_kahn','the_mcguire_sisters','junior_reid','bukka_white','macy_gray','billy_currington','the_tremeloes','ac_dc','little_walter','gorillaz','the_hilltoppers','gil_shaham','little_mix','supertramp','third_world','david_garrett','the_lovin_spoonful','chris_lake','louis_lortie','shakira','vanessa_williams','the_bangles','mark_chesnutt','keith_whitley','roxy_music','betty_hutton','cheryl_cole','the_notorious_b_i_g_','kesha','boston','gerald_moore','heinrich_schiff','joy_division','kenny_loggins','nilsson','counting_crows','simply_red','aaron_tippin','sweet','sugababes','florida_georgia_line','lonnie_mack','jacob_miller','the_police','the_chordettes','fats_domino','willy_deville','bob_crosby','roy_clark','dinu_lipatti','alison_krauss','frankie_avalon','ella_mae_morse','linton_kwesi_johnson','glenn_frey','rudy_vall_e_his_connecticut_yankees','dj_snake','toots_the_maytals','annie_lennox','paramore','john_browning','sister_sledge','julian_lloyd_webber','kanye_west','christopher_cross','professor_longhair','flume','sophie_tucker','third_eye_blind','beyonc_','weezer','kelly_clarkson','take_that','ludwig_van_beethoven','the_cars','george_michael','chaka_khan','arty','steel_pulse','bunny_wailer','the_troggs','nick_lucas','keb_mo_','rise_against','mickey_gilley','poison','debbie_gibson','kim_wilde','sammy_davis_jr','jonas_brothers','cage_the_elephant','son_house','birth_control','faithless','cher','seether','new_kids_on_the_block','rage','richie_spice','sam_smith','the_marvelettes','the_jackson_5','bobbie_gentry','macklemore_ryan_lewis','s_club_7','billy_jones_ernest_hare','arcana','blasterjaxx','will_smith','ray_miller','sugar_ray','ziggy_marley_the_melody_makers','aaron_copland','diddy','daughtry','beach_house','the_dillinger_escape_plan','lonestar','foreigner','lionel_richie','roberta_flack','the_carter_family','demi_lovato','joseph_haydn','sander_van_doorn','underworld','deborah_cox','the_grass_roots','bananarama','iggy_azalea','3_doors_down','the_partridge_family','lead_belly','johnny_long','jagged_edge','the_derek_trucks_band','eddie_fisher','pablo_casals','the_original_dixieland_jazz_band','miley_cyrus','john_powell','the_black_eyed_peas','edvard_grieg','maurizio_pollini','nathan_milstein','twenty_one_pilots','rudolf_firku_n_','nine_inch_nails','the_four_aces','jimmy_rogers','salt_n_pepa','yellow_claw','the_strokes','bobby_brown','juelz_santana','staind','bj_rn_ulvaeus_benny_andersson','the_j_geils_band','the_muppets','gwen_stefani','fedde_le_grand','imagine_dragons','leo_sayer','the_animals','the_b_52_s','furry_lewis','ciara','larry_clinton','dire_straits','pato_banton_the_reggae_revol','goodie_mob','disclosure','georges_bizet','sonny_terry_brownie_mcghee','jim_croce','nelson','jason_derulo','guy_mitchell','the_fontane_sisters','sonny_boy_williamson_i','mirah','jimmy_rushing','john_michael_montgomery','michael_nesmith','george_clinton','burt_bacharach','v6','slim_thug','belinda_carlisle','philip_glass','slade','pete_townshend','oingo_boingo','andrew_lloyd_webber','the_collectors','boy_george','utada_hikaru','mel_torm','diana_krall','melanie_c','john_hammond','peter_green_splinter_group','trouble','g_unit','ferlin_husky','arcade_fire','latino','krayzie_bone','man','dave_clark_five','the_stone_roses','young_jeezy','blood_sweat_tears','kumikameli','dmx','ice_cube','eagles','jill_scott','xtc','peggy_march','michael_bubl','raimon','two_mix','dulce_pontes','the_unseen','hank_locklin','the_notorious_b_i_g','bumblefoot','the_busters','rick_ross','tegan_and_sara','skeeter_davis','curtis_mayfield','sade','wings','lorrie_morgan','saga','a_r_rahman','martha_wainwright','nas','will_i_am','kirsty_maccoll','angel','dave_davies','iggy_pop','jojo','sammy_hagar','ray_davies','and_one','neil_young','mikael_wiehe','the_cardigans','cage','dottie_west','keri_hilson','johnny_hallyday','bill_nelson','fifteen','deen','bobby_v','lil_yachty','too_hort','bernard_lavilliers','hank_thompson','the_chieftains','daryl_hall','antonio_carlos_jobim','aventura','the_the','van_morrison','wynonna_judd','gomez','charles_aznavour','m83','gnr','all','emmylou_harris','lee_hazlewood','mew','uriah_heep','yoko_ono','abw_rts','john_legend','d12','kitty_wells','timbiriche','shel_silverstein','cam_ron','rosanne_cash','2_chainz','tricky','8ball_mjg','flatt_scruggs','bill_anderson','emil_ana_torrini','ufo','mos_def','danzig','juan_gabriel','common','raekwon','france_gall','nicole_scherzinger','r_yksopp','sammie','lena_horne','david_byrne','paul_williams','josh_groban','the_gathering','frank_boeijen','scooter','steve_wariner','mika','pete_seeger','tex_ritter','warrant','porter_wagoner','field_music','three_6_mafia','jim_jones','daniel_o_donnell','brentalfloss','wyclef_jean','hey','bizzy_bone','the_mccalmans','blues_traveler','massive_attack','woody_guthrie','art_garfunkel','andrea_bocelli','david_crosby','dream','soul_asylum','natalie_merchant','shawn_colvin','jonny_lang','funeral_for_a_friend','boz_scaggs','example','lionel_hampton','the_tubes','marc_anthony','good_riddance','moonlight','marc_almond','rza','die_rzte','rbd','alejandro_fern_ndez','wanda_jackson','lara_fabian','julio_iglesias','jeff_beck','peabo_bryson','no_fun_at_all','prong','canibus','krs_one','u_s_bombs','trust','stonewall_jackson','jos_feliciano','m_a','polysics','n_e_r_d','sesame_street','lio','myl_ne_farmer','iris_dement','lily_allen','spoken','architects','jack_johnson','molly_hatchet','cypress_hill','future','the_nits','per_gessle','live','beau','deana_carter','lil_flip','fran_oise_hardy','billy_ray_cyrus','pepper','run_d_m_c','levon_helm','insane_clown_posse','stars','jean_michel_jarre','thunder','juanes','simple_plan','method_man','smash_mouth','meek_mill','fat_joe','kenny_wayne_shepherd','jhen_aiko','the_manhattan_transfer','joss_stone','cee_lo_green','tyrese','charlotte_martin','rodney_crowell','acappella','die_prinzen','pentatonix','abney_park','smooth_mcgroove','the_magnetic_fields','the_nylons','wise_guys','coil','do_as_infinity','lords_of_acid','the_church','chris_rea','jota_quest','miguel_bos','gang_starr','masta_ace','brand_new','mac_miller','cathedral','corrosion_of_conformity','country_joe_mcdonald','eric_johnson','grateful_dead','janis_joplin','john_miles','king_gizzard_the_lizard_wizard','ted_nugent','combichrist','alkaline_trio','anathema','angus_julia_stone','anna_ternheim','anthony_phillips','steve_hackett','aviators','banda_calypso','blue_stahli','the_boys','capital_inicial','city_and_colour','colin_hay','collective_soul','dashboard_confessional','david_rovics','david_usher','die_toten_hosen','funny_van_dannen','dirty_heads','tech_n9ne','elisa','emmerson_nogueira','engenheiros_do_hawaii','eric_bibb','maria_muldaur','panic_at_the_disco','punchline','godsmack','self','heideroosjes','sinner','heather_nova','hoobastank','quietdrive','hyde','jaguares','bert_jansch','jonatha_brooke','joni_mitchell','the_band','josh_garrels','josh_woodward','william_fitzsimmons','katie_melua','jamie_cullum','kristin_hersh','kt_tunstall','legi_o_urbana','the_zombies','francesco_de_gregori','m_ward','beth_orton','magnum','motorpsycho','marillion','jars_of_clay','mason_jennings','matt_nathanson','matthew_good','edguy','gamma_ray','minus_the_bear','mohsen_namjoo','nerina_pallot','never_shout_never','regina_spektor','passenger','paul_kelly','the_style_council','peter_hammill','phantom_planet','phil_keaggy','richard_thompson','said_the_whale','samsas_traum','senses_fail','sevendust','seventh_day_slumber','joan_baez','sister_hazel','slightly_stoopid','sophie_zelmani','suzanne_vega','tatiana','teoman','the_choir','charlotte_church','darlene_zschech','the_front_bottoms','the_maine','the_white_buffalo','little_big_town','threshold','tourniquet','everything_but_the_girl','vertical_horizon','vonda_shepard','warren_zevon','sarah_brightman','blackfoot','black_label_society','z_lia_duncan','2','alejandro_lerner','beth_nielsen_chapman','mercury_rev','brian_wilson','barenaked_ladies','carbon_leaf','celtic_woman','hayley_westenra','crowded_house','delta_goodrem','elbow','resurrection_band','nancy_wilson','janis_ian','jann_arden','jill_sobule','jos_augusto','xuxa','k_d_lang','kim_carnes','los_lobos','mandy_moore','marc_cohn','maureen_mcgovern','melissa_manchester','patti_labelle','helene_fischer','laura_pausini','ivan_lins','thal_a','mike_the_mechanics','paul_carrack','natasha_st_pier','michael_mcdonald','olivia','dizzee_rascal','sam_phillips','serge_gainsbourg','jane_birkin','luis_miguel','sondre_lerche','stan_ridgway','susan_boyle','mike_oldfield','ces_ria_vora','agonoize','funker_vogt','god_module','hocico','nachtmahr','suicide_commando','alejandro_escovedo','southside_johnny_the_asbury_jukes','broken_social_scene','vigilantes_of_love','billy_bragg','wilco','frank_black_and_the_catholics','blue_rodeo','brandi_carlile','patty_griffin','calexico','cass_mccombs','chris_knight','conor_oberst','corb_lund','cowboy_junkies','cracker','cross_canadian_ragweed','dave_alvin','drive_by_truckers','eleni_mandell','lucinda_williams','fred_eaglesmith','dar_williams','the_jayhawks','lana_del_rey','tim_o_brien','hank_williams_iii','james_mcmurtry','joe_henry','john_stewart','josh_ritter','lambchop','nanci_griffith','norma_jean','lyle_lovett','matthew_ryan','my_morning_jacket','neko_case','the_new_pornographers','blue_october','okkervil_river','old_97_s','ray_wylie_hubbard','richmond_fontaine','robert_earl_keen','rocky_votolato','ryan_adams','whiskeytown','son_volt','steve_earle','the_avett_brothers','the_bottle_rockets','the_felice_brothers','arlo_guthrie','the_handsome_family','the_mavericks','ten_years_after','the_walkabouts','todd_snider','vic_chesnutt','iron_wine','wovenhand','x','big_audio_dynamite','globe','carter_the_unstoppable_sex_machine','allison_moorer','front_line_assembly','the_national','the_fall','public_image_ltd','public_enemy','wire','a_tribe_called_quest','de_la_soul','aesop_rock','buck_65','caparezza','childish_gambino','the_roots','colbie_caillat','big_sean','dj_gruff','tyler_the_creator','dokken','fun_lovin_criminals','talib_kweli','jane_air','k_i_z','kid_cudi','jedi_mind_tricks','celph_titled','lupe_fiasco','bun_b','scarface','ghostface_killah','robyn','rehab','swollen_members','styles_p','the_streets','wale','joe_budden','tank','10_years','36_crazyfists','apocalyptica','nina_hagen','anastacia','black_stone_cherry','blindside','breaking_benjamin','bring_me_the_horizon','bullet_for_my_valentine','cave_in','chevelle','d_espairsray','death_angel','deftones','demon_hunter','demon','devin_townsend_project','devin_townsend','doa','dir_en_grey','disturbed','dope','drowning_pool','eighteen_visions','entombed','faith_no_more','fear_factory','fightstar','five_finger_death_punch','finger_eleven','flyleaf','grinspoon','guano_apes','h_blockx','halestorm','hamlet','helmet','bt','ill_ni_o','in_flames','in_this_moment','him','j_b_o','katatonia','killswitch_engage','xzibit','lacuna_coil','phish','limp_bizkit','living_colour','viikate','marilyn_manson','megaherz','falco','melvins','monster_magnet','mushroomhead','nonpoint','soil','otep','p_o_d','powerman_5000','primus','project_86','red','kris_allen','rob_zombie','ozzy_osbourne','rollins_band','saliva','sepultura','shihad','skillet','skindred','slipknot','smile_empty_soul','danielson','soilwork','sonic_syndicate','static_x','stone_sour','taproot','the_notwist','the_word_alive','theory_of_a_deadman','therapy','los_tucanes_de_tijuana','manu_chao','volbeat','zebrahead','hed_p_e','and_you_will_know_us_by_the_trail_of_dead','10_000_maniacs','311','77s','yes','david_lee_roth','hillsong','afi','adam_sandler','afterhours','hawkwind','all_about_eve','all_time_low','allison_crowe','amanda_palmer','american_music_club','amplifier','robert_wyatt','anberlin','andrew_bird','ani_difranco','apoptygma_berzerk','apulanta','arab_strap','joseph_arthur','tom_rosenthal','ash','asian_kung_fu_generation','poets_of_the_fall','babas_nicos','bayside','beatsteaks','ben_folds','ben_folds_five','ben_harper','better_than_ezra','bettie_serveert','big_country','big_head_todd_and_the_monsters','big_sugar','billy_talent','today_is_the_day','red_flag','black_rebel_motorcycle_club','megadeth','blonde_redhead','bob_mould','bodeans','bowling_for_soup','buck_tick','butch_walker','butthole_surfers','caf_tacvba','cake','camper_van_beethoven','carmen_consoli','mario_venuti','franco_battiato','catherine_wheel','catupecu_machu','cem_adrian','john_cale','charlie_brown_jr','nena','chumbawamba','clutch','cl','cmx','coheed_and_cambria','cold_war_kids','travis','coma','concrete_blonde','mint_condition','copeland','crash_test_dummies','joe_jackson','cristian_castro','curve','dada','daniel_amos','daniel_johnston','dave_matthews_band','burning_heads','david_gray','david_sylvian','deacon_blue','deerhoof','del_amitri','dinosaur_jr','dirty_projectors','draco_rosa','duncan_sheik','jeremy_camp','edwyn_collins','eels','nightwish','element_of_crime','embrace','enter_shikari','ulver','everclear','everlast','eyeshine','dio','faust_o','feeder','atmosphere','filter','firewater','fishbone','fountains_of_wayne','four_year_strong','steve_green','fresno','gang_of_four','good_charlotte','blood_on_the_dance_floor','graham_coxon','melissa_etheridge','tony_joe_white','guided_by_voices','robert_pollard','guster','elliott_smith','hedley','hole','hollywood_undead','hot_chip','l_arc_en_ciel','ian_brown','idlewild','jimmy_eat_world','fish','ingrid_michaelson','inme','inspiral_carpets','raf','james','jean_leloup','weird_al_yankovic','jeff_buckley','john_frusciante','dr_john','pj_harvey','jonathan_coulton','juliana_hatfield','julieta_venegas','k_s_choice','kaizers_orchestra','kargo','kasabian','keane','kevin_coyne','kevin_devine','kevin_max','rich_mullins','trooper','suzy_bogguss','kill_hannah','kisp_l_s_a_borz','kult','my_life_with_the_thrill_kill_kult','kutless','la_barranca','la_ley','lao_che','lech_janerka','les_cowboys_fringants','les_fatals_picards','les_rita_mitsouko','sparks','lifehouse','lisa_germano','ed_harcourt','lisa_loeb','liz_phair','local_h','lost_dogs','lostprophets','love_and_rockets','lucybell','lulu_santos','gabriel_o_pensador','adam_lambert','madrugada','mancha_de_rolando','manchester_orchestra','mando_diao','foetus','mark_lanegan','matthew_sweet','max_mo_park','mayday_parade','meat_puppets','men_without_hats','meshell_ndegeocello','midnight_oil','dance_gavin_dance','molotov','ov7','monkey_majik','suede','fernando_ortega','motion_city_soundtrack','mudhoney','mutemath','mercyme','m_o_morta','natalia_lafourcade','natewantstobattle','needtobreathe','split_enz','sum_41','no_te_va_gustar','noir_d_sir','t_tes_raides','o_rappa','o_a_r','ocean_colour_scene','omul_cu_obolani','one_ok_rock','2raumwohnung','our_lady_peace','pain','panda','parokya_ni_edgar','pato_fu','paul_westerberg','pere_ubu','pete_yorn','peter_murphy','placebo','plain_white_t_s','pop_will_eat_itself','porcupine_tree','powderfinger','cat_power','casting_crowns','primal_scream','m_tley_cr_e','the_used','raimundos','mark_knopfler','mark_kozelek','danko_jones','relient_k','raffi','renaud','richard_hawley','rickie_lee_jones','the_shins','rilo_kiley','robyn_hitchcock','mose_allison','roy_harper','rucka_rucka_ali','rx_bandits','saez','samiam','sarah_slean','say_anything','scout_niblett','screaming_females','shannon_wright','silverchair','sin_ad_o_connor','siouxsie_and_the_banshees','sixpence_none_the_richer','skank','skunk_anansie','sleater_kinney','sloan','social_distortion','sophie_hunger','e_40','steve_wynn','subsonica','joe_walsh','super_furry_animals','superchunk','supergrass','swervedriver','switchfoot','dido','takida','taking_back_sunday','teenage_fanclub','w_a_s_p','the_afghan_whigs','the_apples_in_stereo','the_ataris','smoking_popes','the_bluetones','the_breeders','the_cat_empire','the_charlatans_uk','the_clarks','guy_clark','the_comsat_angels','the_connells','the_coral','the_cribs','the_cult','bobby_o','the_mission','blue_yster_cult','the_dandy_warhols','the_dear_hunter','the_decemberists','the_early_november','thievery_corporation','the_fratellis','the_gaslight_anthem','jim_brickman','falling_up','the_hives','the_innocence_mission','the_jazz_butcher','the_jesus_lizard','the_lemonheads','babyshambles','the_living_end','the_matrixx','the_mother_hips','the_mountain_goats','the_muffs','the_pillows','the_posies','the_presidents_of_the_united_states_of_america','the_rasmus','the_raveonettes','the_saints','the_samples','bad_religion','the_smithereens','the_soundtrack_of_our_lives','the_tea_party','mayday','the_triffids','the_vines','the_violet_burning','the_wallflowers','testament','the_divine_comedy','third_day','thrice','tindersticks','tism','tit_s','toad_the_wet_sprocket','tocotronic','tom_mcrae','tori_amos','tracy_chapman','trashcan_sinatras','tre_allegri_ragazzi_morti','tub_ring','unkle','unwritten_law','uverworld','vast','verdena','veruca_salt','face_to_face','virus','voltaire','we_the_kings','the_kooks','lindisfarne','seals_crofts','andy_partridge','xutos_pontap_s','yellowcard','yup','leevi_and_the_leavings','zo','zucchero','z','ebnem_ferah','air','alice','boards_of_canada','brian_eno','burzum','daniel_lanois','enigma','juana_molina','lisa_gerrard','nox_arcana','renard','schiller','sigur_r_s','steven_wilson','swans','wolfgun','xiu_xiu','michael_johnson','montgomery_gentry','the_stanley_brothers','john_waite','shelby_lynne','judy_collins','burl_ives','the_irish_rovers','david_wilcox','devendra_banhart','doc_watson','bill_monroe','michael_martin_murphey','gordon_bok','asleep_at_the_wheel','the_browns','nana_mouskouri','jerry_jeff_walker','steve_goodman','malcolm_holcombe','malvina_reynolds','odetta','tom_paxton','strawbs','phil_ochs','harry_chapin','ramblin_jack_elliott','roger_mcguinn','gene_clark','mat_kearney','the_brothers_four','tom_russell','townes_van_zandt','uncle_dave_macon','delbert_mcclinton','john_hiatt','justin_townes_earle','mark_erelli','over_the_rhine','steve_forbert','manfred_mann_s_earth_band','mot_rhead','rudimentary_peni','illapu','inti_illimani','quilapay_n','v_ctor_jara','skylark','adam_green','cold_chisel','guy_sebastian','jefferson_starship','the_alan_parsons_project','ali_project','modern_talking','animal_collective','banco_del_mutuo_soccorso','ben_lee','bryan_ferry','buffy_sainte_marie','colin_blunstone','cursive','elysian_fields','emerson_lake_palmer','gino_vannelli','g_rard_manset','hot_dad','marina_and_the_diamonds','ismo_alanko','kansas','kari_peitsamo','laibach','laurie_anderson','puhdys','na_o_zumbi','roger_waters','rush','the_walker_brothers','hilltop_hoods','wolfgang_ambros','erste_allgemeine_verunsicherung','jacques_brel','rainhard_fendrich','tom_waits','adrian_belew','anne_clark','can','captain_beefheart_and_the_magic_band','deine_lakaien','devo','einst_rzende_neubauten','frank_zappa','goethes_erben','wishbone_ash','death_cab_for_cutie','antony_and_the_johnsons','jandek','nevermore','king_crimson','king_missile','the_residents','steeleye_span','vampire_rodents','the_walkmen','dog_fashion_disco','freak_kitchen','sigh','children_of_bodom','soft_machine','ara_ketu','asa_de_guia','banda_eva','ivete_sangalo','chiclete_com_banana','daniela_mercury','alejandro_sanz','timbalada','juan_luis_guerra','daddy_yankee','alceu_valen_a','luiz_gonzaga','matia_bazar','axelle_red','barbara','benny_neyman','gigi_d_agostino','jacques_higelin','caetano_veloso','gal_costa','jorge_ben','die_flippers','nicole','angra','reinhard_mey','wolf_biermann','florent_pagny','hannes_wader','tienne_daho','henri_salvador','f_lix_leclerc','daniel_lavoie','gerhard_sch_ne','g_lben_ergen','georg_kreisler','herbert_gr_nemeyer','herman_van_veen','hildegard_knef','marlene_dietrich','iu','jos_luis_rodr_guez','juliette_gr_co','klaus_hoffmann','konstantin_wecker','saltatio_mortis','luigi_tenco','maria_beth_nia','adriana_calcanhotto','marie_lafor_t','marius_m_ller_westernhagen','mina','no_l_coward','pippo_pollina','rita_lee','os_mutantes','rita_pavone','roger_whittaker','al_bano_romina_power','salvatore_adamo','simone','s_rgio_godinho','udo_j_rgens','udo_lindenberg','ulrich_roski','zaz','z_ramalho','fagner','dith_piaf','duelo','espinoza_paz','fidel_rueda','la_firma','la_arrolladora_banda_el_lim_n','voz_de_mando','sergio_vega','fool_s_garden','waltari','of_montreal','pierre_lapointe','rufus_wainwright','loudon_wainwright_iii','sufjan_stevens','machine_gun_kelly','francesco_guccini','le_orme','lucio_dalla','michel_fugain','al_jarreau','carmen_mcrae','javier_sol_s','harry_connick_jr','bap','cradle_of_filth','amorphis','avatar','bathory','behemoth','borknagar','countess','cruachan','darkthrone','hate','destruction','dimmu_borgir','eisregen','enslaved','finntroll','fates_warning','graveworm','impaled_nazarene','sentenced','king_diamond','kreator','lord_belial','marduk','mercyful_fate','stick_to_your_guns','moonspell','as_i_lay_dying','nunslaughter','rotting_christ','samael','sandy_denny','skyforger','sodom','cannibal_corpse','exodus','atreyu','theatres_des_vampires','wizard','transmetal','venom','belphegor','the_crown','moya_brennan','todd_rundgren','clay_walker','andrew_peterson','lynn_anderson','david_crowder_band','pam_tillis','norah_jones','rhonda_vincent','jamey_johnson','plumb','j_j_cale','new_riders_of_the_purple_sage','joe_diffie','kasey_chambers','leon_russell','jack_greene','the_string_cheese_incident','ystein_sunde','stephen_stills','cancerslug','robert_plant','alvin_lee','beth_hart','jimmy_buffett','billy_s_band','bunbury','nacho_vegas','calogero','georges_brassens','canned_heat','charlie_louvin','colin_james','cuby_blizzards','dick_annegarn','edoardo_bennato','eva_cassidy','gil_scott_heron','glenn_hughes','deep_purple','connie_smith','iva_zanicchi','izzy_stradlin','j_karjalainen','jack_bruce','leonard_cohen','joan_armatrading','joan_osborne','john_martyn','rio_reiser','larry_carlton','madeleine_peyroux','bruce_cockburn','kate_anna_mcgarrigle','mavis_staples','noa','ralph_mctell','renato_carosone','richie_kotzen','robben_ford','roberto_carlos','erasmo_carlos','robin_trower','rory_block','roy_buchanan','sandra_mihanovich','savoy_brown','shirley_horn','siniestro_total','slank','the_fabulous_thunderbirds','the_seatbelts','the_tragically_hip','mike_jones','trophy_scars','caravan','velhas_virgens','walter_trout','gov_t_mule','bar_o_vermelho','blue_cheer','ian_hunter','david_leb_n','de_palmas','eugenio_finardi','extreme','foghat','george_thorogood_the_destroyers','great_white','guardian','jethro_tull','ian_anderson','david_knopfler','steppenwolf','dave_edmunds','lynyrd_skynyrd','crosby_stills_nash','raul_seixas','the_poodles','musiq_soulchild','shocking_blue','nick_lowe','the_black_crowes','traffic','widespread_panic','co','alberto_cortez','joan_sebastian','ana_gabriel','gilberto_santa_rosa','rub_n_blades','v_ctor_manuelle','celia_cruz','luis_fonsi','nek','dr_feelgood','astrud_gilberto','benito_di_paula','brazzaville','sacha_distel','chico_buarque','elis_regina','milton_nascimento','faf_de_bel_m','nikka_costa','tim_maia','gilberto_gil','lisa_ekdahl','joyce','maria_rita','nara_le_o','nouvelle_vague','paulinho_moska','wilson_simonal','14_bis','arnaldo_antunes','biquini_cavad_o','cidade_negra','cpm_22','c_ssia_eller','os_paralamas_do_sucesso','guilherme_arantes','ira','lob_o','nenhum_de_n_s','djavan','rog_rio_skylab','roupa_nova','ultraje_a_rigor','kj_52','amado_batista','chit_ozinho_xoror','jo_o_paulo_daniel','leandro_leonardo','leonardo','odair_jos','kaiser_chiefs','kula_shaker','lightning_seeds','pulp','the_proclaimers','dying_fetus','napalm_death','nile','pathology','hilary_duff','badly_drawn_boy','federico_salvatore','i_gufi','zachary_richard','stan_rogers','moxy_fr_vous','poco','la_bottine_souriante','stompin_tom_connors','bersuit_vergarabat','las_pastillas_del_abuelo','george_lam','altan','clannad','blackmore_s_night','capercaillie','celtic_thunder','eluveitie','powerwolf','gaelic_storm','an_na','jon_anderson','the_dubliners','loreena_mckennitt','omnia','secret_garden','shaun_davey','roger_daltrey','the_corrs','los_tigres_del_norte','laurent_voulzy','the_kelly_family','wolfe_tones','alan_stivell','heather_alexander','kate_rusby','dropkick_murphys','great_big_sea','fiddler_s_green','heather_dale','runrig','the_waterboys','dougie_maclean','adriano_celentano','alain_chamfort','zazie','hamelen','tazenda','arno','arthur_h','boudewijn_de_groot','charles_trenet','claudio_baglioni','claudio_rocchi','fabrizio_de_andr','dalida','dana_winner','demis_roussos','esther_ofarim','eugenio_bennato','michel_berger','francis_cabrel','maxime_le_forestier','georges_moustaki','gianmaria_testa','gianni_morandi','gigliola_cinquetti','milva','gilbert_b_caud','ginette_reno','giuni_russo','guy_b_art','helena_vondr_kov','hugues_aufray','ivan_graziani','ivano_fossati','jacques_bertin','jean_ferrat','juliane_werding','julien_clerc','los_temerarios','katerine','leny_escudero','mathieu_chedid','luca_barbarossa','l_o_ferr','rosenstolz','marc_lavoine','massimo_bubola','mecano','mia_martini','michel_jonasz','michele_zarrillo','fiorello','nada','mercedes_sosa','nino_d_angelo','patrick_bruel','patty_pravo','pierre_bachelet','rainald_grebe','rapha_l','raphael','richard_anthony','roberto_murolo','ron','stefano_rosso','stephan_eicher','vasco_rossi','yves_duteil','yves_jamait','ang_lica','aaron_carter','barry_louis_polisar','yuri','cri_cri','hevisaurus','juice_leskinen','kidz_bop','mara_maravilha','destroyer','scorpions','obk','duncan_dhu','parry_gripp','sandy_junior','the_verve_pipe','the_verve','the_wiggles','veggietales','newsboys','steven_curtis_chapman','toro_y_moi','medi_val_b_bes','aaron_neville','bethel_music','apologetix','gaither_vocal_band','building_429','chris_tomlin','matt_maher','jerusalem','david_meece','debby_boone','elevation_worship','matt_redman','planetshakers','majesty','jump5','lecrae','michael_w_smith','bride','natalie_grant','the_lads','audio_adrenaline','paul_wilbur','psalmen_voor_nu','sawyer_brown','shane_shane','the_echoing_green','twila_paris','watch_tower_bible_and_tract_society','da_t_r_u_t_h','dc_talk','flame','grits','trip_lee','crystal_lewis','the_cross_movement','tobymac','vico_c','mormon_tabernacle_choir','august_burns_red','black_veil_brides','deliverance','opeth','die_happy','disciple','galactic_cowboys','haste_the_day','living_sacrifice','mastodon','mortification','showbread','labyrinth','stryper','the_devil_wears_prada','underoath','whitecross','petra','huntingtons','mxpx','d_a_d','caedmon_s_call','david_and_the_giants','degarmo_and_key','delirious','don_francisco','five_iron_frenzy','geoff_moore','hawk_nelson','grave','larry_norman','randy_stonehill','monty_python','oomph','oficina_g3','white_heart','rescate','rick_wakeman','la_oreja_de_van_gogh','sanctus_real','fun_people','thousand_foot_krutch','tim_hughes','the_o_c_supertones','4him','billy_gilman','aimee_mann','katharine_mcphee','eros_ramazzotti','z_ro','babbie_mason','bebo_norman','judy_garland','carman','cece_winans','trick_daddy','chris_isaak','cocteau_twins','edyta_g_rniak','enrico_ruggeri','ffh','hanson','hawksley_workman','indigo_girls','irene_grandi','jackie_evancho','joy_electric','kelly_price','mary_mary','israel_houghton','phil_wickham','phillips_craig_dean','roch_voisine','rupaul','gregorian','sarah_connor','sugarland','sweetbox','tarja','the_brian_setzer_orchestra','brian_setzer','badfinger','the_moffatts','the_vandals','trans_siberian_orchestra','roy_drusky','burton_cummings','procol_harum','renaissance','the_pretty_things','twisted_sister','bj_rn_eidsv_g','corvus_corax','schelmish','emilie_autumn','epica','katherine_jenkins','scala_kolacny_brothers','take_6','the_roches','tony_banks','to_e_proeski','lacrimosa','16_volt','bj_rn_rosenstr_m','bob_rivers','cledus_t_judd','frankjavcee','george_formby','ninja_sex_party','paul_and_storm','the_arrogant_worms','tripod','el_cuarteto_de_nos','gwar','knorkator','psychostick','rodgau_monotones','los_palominos','charlie_peacock','jesus_culture','michael_card','tenth_avenue_north','carrie_newcomer','nick_drake','aaron_watson','billy_joe_royal','billy_joe_shaver','charlie_landsborough','chris_ledoux','collin_raye','dan_seals','dave_dudley','hellbillies','ed_bruce','emilio_navaira','jean_shepard','freddie_hart','gary_stewart','gene_watson','gian_giovani','gilberto_gilmar','jason_mraz','ilse_delange','john_prine','jake_owen','wynn_stewart','jim_ed_brown','joe_ely','kid_rock','la_toya_jackson','lit','lita_ford','me_first_and_the_gimme_gimmes','lagwagon','melanie','mickey_newbury','paul_brunelle','paula_fernandes','zez_di_camargo_luciano','randy_rogers_band','reverend_horton_heat','rick_renner','rionegro_solim_es','shooter_jennings','terri_clark','vern_gosdin','webb_wilder','ween','38_special','the_beau_brummels','matanza','clawfinger','acid_drinkers','agnostic_front','biohazard','body_count','d_r_i','municipal_waste','neurosis','nuclear_assault','soziedad_alkoholika','suicidal_tendencies','paragon','mario','inna','belinda','bronco','grupo_bryndis','david_bisbal','ram_n_ayala','grant_lee_phillips','the_veronicas','amr_diab','atb','basshunter','dream_theater','frankie_j','baby_bash','sophie_ellis_bextor','grace_jones','laveerre','silkk_the_shocker','parov_stelar','raffaella_carr','elephant_man','saint_etienne','samantha_fox','selena','super_junior','t_a_t_u','tarkan','judie_tzuke','el_kel_iset','yello','franz_ferdinand','chenoa','lucero','tokio','puffy_amiyumi','wink','obie_trice','mystikal','current_93','dark_sanctuary','rome','lord_of_the_lost','bella_morte','mantus','blutengel','clan_of_xymox','dead_can_dance','death_in_june','diary_of_dreams','diorama','helium_vola','illuminate','l_me_immortelle','lacrimas_profundere','killing_joke','m_nchener_freiheit','otto_dix','project_pitchfork','qntal','sopor_aeternus','the_cr_xshadows','unheilig','welle','yendri','carcass','asphyx','bolt_thrower','darkseed','paradise_lost','tiamat','the_damned','pantera','the_amity_affliction','judas_priest','amon_amarth','alesana','atrocity','autopsy','avulsed','sabaton','misfits','iron_fire','centinex','dagoba','dark_tranquillity','asia','deicide','dethklok','dew_scented','edge_of_sanity','escape_the_fate','heaven_shall_burn','hypocrisy','incantation','jungle_rot','kataklysm','krisiun','macabre','malevolent_creation','meshuggah','misanthrope','morbid_angel','dead_kennedys','necro','pig_destroyer','shadows_fall','sinister','six_feet_under','dream_evil','soulfly','the_black_dahlia_murder','between_the_buried_and_me','therion','vader','whitechapel','attila','emmure','miss_may_i','the_acacia_strain','betontod','broilers','dritte_wahl','ohl','slime','terrorgruppe','b_hse_onkelz','frei_wild','k_rbholz','asp','tokio_hotel','queensr_che','amanda_miguel','arabesque','bad_boys_blue','boyce_avenue','parliament','wu_tang_clan','neoton_fam_lia','teena_marie','bobby_womack','agoraphobic_nosebleed','candlemass','electric_wizard','black_sabbath','theatre_of_tragedy','type_o_negative','marie_fredriksson','luna','marissa_nadler','yo_la_tengo','celldweller','hitomi','big_d_and_the_kids_table','alacranes_musical','k_paz_de_la_sierra','assemblage_23','covenant','die_krupps','kodak_black','front_242','haujobb','in_strict_confidence','le_ther_strip','snog','the_darkness','tanzwut','terminal_choice','velvet_acid_christ','vnv_nation','wumpscut','x_fusion','umbra_et_imago','de_vision','deichkind','eisbrecher','herbie_hancock','ana_moura','macaco','skinny_puppy','ayreon','black_moth_super_rainbow','erykah_badu','cocorosie','de_jeugd_van_tegenwoordig','dj_shadow','e_nomine','kmfdm','flying_lotus','goldfrapp','hanzel_und_gretyl','information_society','mc_frontalot','kraftwerk','ladytron','lamb','milk_inc','mind_in_a_box','ministry','m_m','m_nia','pig','pitchshifter','lil_boosie','master_p','mindless_self_indulgence','buzzcocks','vanilla_ice','milie_simon','gianna_nannini','pinback','the_birthday_massacre','archive','99_posse','bloc_party','morcheeba','origa','paul_kalkbrenner','tina_arena','dover','melotron','owl_city','kamelot','greeley_estates','hawthorne_heights','joan_of_arc','saves_the_day','thursday','transit','fairport_convention','maggie_reilly','joan_manuel_serrat','e_rotic','the_scene','sandra','amon_d_l_ii','circa_survive','love_solfege','caliban','tall_dwarfs','van_der_graaf_generator','death_grips','the_fiery_furnaces','am_lia_rodrigues','cristina_branco','jos_afonso','katia_guerreiro','ney_matogrosso','madredeus','mariza','gipsy_kings','mal','aleks_syntek','ni_a_pastori','rosario','al_stewart','amos_lee','andr_s_calamaro','ane_brun','asa','editors','catie_curtis','chrystian_ralf','clueso','eddi_reader','eddie_from_ohio','ellis_paul','frank_turner','estampie','ferdi_tayfur','fito_p_ez','luis_alberto_spinetta','gabriella_ferri','gigi','greg_brown','g_ksel','lando_fiorini','india_arie','jack_savoretti','anne_grete_preus','jarom_r_nohavica','joe_purdy','john_wesley_harding','josh_rouse','karel_kryl','v_tor_ramil','lars_winnerb_ck','laura_marling','llu_s_llach','los_chalchaleros','luka_bloom','malicorne','mark_heard','martin_carthy','nic_jones','le_n_gieco','mijares','nuova_compagnia_di_canto_popolare','ola_magnell','thin_lizzy','ray_lamontagne','ron_sexsmith','rosana','silvio_rodr_guez','stef_bos','sun_kil_moon','tanita_tikaram','the_incredible_string_band','thea_gilmore','tina_dico','victor_leo','v_rttin','ge_aleksandersen','i_brahim_tatl_ses','ektomorf','elvenking','ensiferum','falconer','feuerschwanz','in_extremo','korpiklaani','leaves_eyes','letzte_instanz','m_go_de_oz','saurom','schandmaul','skyclad','subway_to_sally','suidakra','t_r','icehouse','bomb_the_music_industry','the_real_mckenzies','54_40','armored_saint','alexz_johnson','bar_man_o','ezginin_g_nl','galija','sts','h_kan_hellstr_m','james_blunt','kazik','mewithoutyou','michel_polnareff','ovidi_montllor','rasputina','shearwater','gerry_rafferty','steam_powered_giraffe','the_saw_doctors','ty_segall','tyrone_wells','avi_es_do_forr','grimskunk','sinik','vitaa','kenza_farah','sexion_d_assaut','aliz_e','henri_tachan','jenifer','m_pokora','indochine','brainstorm','con_funk_shun','funkadelic','lena_park','neffa','ugk','suburban_legends','mai_kuraki','cherry_poppin_daddies','electric_six','los_straitjackets','the_69_eyes','the_angels','the_haunted','the_hellacopters','the_kills','thee_oh_sees','white_denim','zabranjeno_pu_enje','ol_dirty_bastard','kurupt','spice_1','brotha_lynch_hung','chamillionaire','paul_wall','trae','club_dogo','mc_eiht','royce_da_5_9','geto_boys','the_diplomats','ice_t','2_live_crew','xv','mobb_deep','c_murder','tru','lil_keke','project_pat','tha_dogg_pound','esham','twiztid','erick_sermon','big_tymers','kate_nash','the_cramps','nekromantix','tsol','ace_frehley','hardcore_superstar','harem_scarem','house_of_lords','kingdom_come','l_a_guns','mr_big','pink_cream_69','quiet_riot','riot','ratt','tnt','backyard_babies','ultima_thule','europe','hanoi_rocks','mott_the_hoople','smokie','suzi_quatro','haemorrhage','aline_barros','bruna_karla','kirk_franklin','minist_rio_koinonya_de_louvor','artrosis','closterkeller','indica','sirenia','trail_of_tears','tristania','within_temptation','bauhaus','mono_inc','pansy_division','xandria','immortal_technique','agathocles','rotten_sound','the_locust','anthrax','devildriver','lamb_of_god','machine_head','parkway_drive','pro_pain','throwdown','vicious_rumors','screaming_trees','cuisillos','intocable','pesado','la_mafia','marco_antonio_sol_s','los_bukis','andrew_w_k','april_wine','axel_rudi_pell','b_z','tak_matsumoto','barricada','bijelo_dugme','blaze_bayley','bonfire','bruce_dickinson','buckcherry','budgie','buitres','jorn','doro','enuff_z_nuff','gentle_giant','girlschool','golden_earring','gotthard','nazareth','a_day_to_remember','jefferson_airplane','joe_satriani','ken_hensley','kim_mitchell','king_s_x','kotiteollisuus','la_renga','lee_aaron','lordi','michael_schenker_group','mustasch','night_ranger','omega','parni_valjak','paul_gilbert','popeda','skid_row','tankcsapda','the_bronx','the_donnas','all_that_remains','triumph','umphrey_s_mcgee','y_t','ziggy','sfdk','7_seconds','aiden','alphaville','black_flag','slayer','circle_jerks','ritchie','converge','every_time_i_die','hatebreed','nomeansno','rancid','memphis_may_fire','nofx','propagandhi','tankard','screeching_weasel','sick_of_it_all','silverstein','two_steps_from_hell','faun','accept','the_frames','andromeda','annihilator','anvil','artillery','avenged_sevenfold','axxis','blind_guardian','vanden_plas','grave_digger','dragonforce','edenbridge','damien_jurado','exciter','firewind','halford','hammerfall','helloween','helstar','iced_earth','jag_panzer','machinae_supremacy','manowar','metal_church','morgana_lefay','mudvayne','nocturnal_rites','overkill','primal_fear','rebellion','running_wild','corey_hart','savatage','saxon','steve_vai','tad_morose','tarot','tierra_santa','trivium','turmion_k_til_t','u_d_o','virgin_steele','voivod','warcry','yngwie_malmsteen','zion_lennox','sido','mc_chris','assalti_frontali','kool_keith','ayumi_hamasaki','az','bahh_tee','bassi_maestro','revocation','blumentopf','brockhampton','bts','bushido','vinnie_paz','chakuza','cheek','cro','arc_ngel','alexis_fido','dargen_d_amico','the_coup','def_con_dos','die_fantastischen_vier','dom_no','donguralesko','epmd','kool_savas','fettes_brot','fronda','mc_solaar','pyhimys','kaaris','kollegah','kontra_k','k_k','l_o_c','logic','jerry_rivera','murs','angie_stone','namie_amuro','anthony_hamilton','lyfe_jennings','bl_f','o_s_t_r','paluch','parazi_ii','porta','bleeding_through','prinz_pi','rasmentalism','xavier_naidoo','sage_francis','stupeflip','young_thug','tego_calder_n','fifth_harmony','jay_chou','blitzkid','zumbis_do_espa_o','deer_tick','half_man_half_biscuit','hayden','club_8','grandaddy','jens_lekman','kent','keren_ann','los_campesinos','nellie_mckay','china_crisis','prefab_sprout','the_clientele','the_lucksmiths','bell_x1','british_sea_power','car_seat_headrest','deerhunter','dr_dog','elf_power','frightened_rabbit','fugazi','fury_in_the_slaughterhouse','julie_doiron','tinashe','la_habitaci_n_roja','margot_the_nuclear_so_and_so_s','matt_pond_pa','metric','mike_doughty','mother_mother','piebald','quasi','rheostatics','sebadoh','spoon','starflyer_59','stephen_malkmus','stereolab','ted_leo_and_the_pharmacists','the_appleseed_cast','the_faint','the_go_betweens','the_pineapple_thief','the_undertones','tronic','chris_de_burgh','mass_hysteria','angelo_branduardi','gigi_d_alessio','i_muvrini','back_number','boa','claris','crystal_kay','zard','gackt','garnet_crow','girls_generation','kat_tun','koda_kumi','kotoko','lisa','maaya_sakamoto','masami_okui','mr_children','news','shinee','w_inds','yui','yumi_matsutoya','the_high_lows','sid','abbey_lincoln','anna_maria_jopek','cassandra_wilson','dianne_reeves','fred_buscaglione','jane_monheit','zor_n','kraan','laura_fygi','michael_franks','natalino_otto','quartetto_cetra','scott_bradlee_s_postmodern_jukebox','stacey_kent','the_flower_kings','ronnie_von','brown_eyed_girls','ahmet_kaya','alejandra_guzm_n','ana_carolina','alcione','el_chapo_de_sinaloa','gustavo_cerati','soda_stereo','jenni_rivera','joaqu_n_sabina','los_fabulosos_cadillacs','abel_pintos','ana_bel_n','aterciopelados','camilo_sesto','david_demar_a','gian_marco','menudo','ricardo_arjona','sabroso','v_ctor_manuel','las_pelotas','ariel_pink','leehom_wang','jolin_tsai','darkest_hour','kalmah','nightrage','eppu_normaali','the_outfield','no_use_for_a_name','pennywise','callejon','d_f_c','our_last_night','exaltasamba','beth_carvalho','jo_o_bosco','marina_lima','marisa_monte','nando_reis','natiruts','ra_a_negra','s_pra_contrariar','zeca_pagodinho','andr_hazes','de_dijk','arena','iq','sol_invictus','new_found_glory','adam_ant','berlin','hoodoo_gurus','ultravox','nik_kershaw','squeeze','the_aquabats','the_fixx','beat_crusaders','cows','conjunto_primavera','peter_and_the_test_tube_babies','sham_69','the_adicts','the_analogs','instalok','jacek_kaczmarski','przemys_aw_gintrowski','ada_band','agnetha_f_ltskog','ajda_pekkan','al_bano','alex_ubago','alison_moyet','alunni_del_sole','anna_oxa','bajm','barclay_james_harvest','blue_system','brunner_brunner','candan_er_etin','christian_bautista','clay_aiken','clifford_t_ward','daniel','don_backy','jesse_mccartney','emma','marcella_bella','giorgio_gaber','guus_meeuwis','heinz_rudolf_kunze','john_farnham','ian_thomas','i_n_karaca','jennifer_rush','jo_vally','john_fogerty','julian_lennon','k3','kid_abelha','labv_l_gais_tips','l_vi','lea_salonga','les_wampas','magnus_uggla','mango','maria_mena','massimo_ranieri','max_gazz','michael_learns_to_rock','mietta','mustafa_sandal','nil_fer','peter_frampton','pr_ta_v_tra','pur','rettore','ricchi_e_poveri','rob_de_nijs','sara_bareilles','sasha','sertab_erener','sezen_aksu','stadio','stephen_sondheim','tamara','team_starkid','toto_cutugno','umberto_tozzi','herman_brood','wanessa','zen_caf','bonanza_banzai','bodyjar','bracket','frenzal_rhomb','goldfinger','the_wonder_years','useless_id','camel','hombres_g','leo_jaime','neal_morse','spock_s_beard','new_trolls','opus','piersi','premiata_forneria_marconi','superbus','zmelkoow','boysetsfire','hot_water_music','new_model_army','the_monochrome_set','big_big_train','avantasia','dark_moor','dreamtale','freedom_call','mystic_prophecy','nightmare','rhapsody_of_fire','royal_hunt','sonata_arctica','stratovarius','symphony_x','vision_divine','the_wildhearts','armia','evergrey','lana_lane','nektar','pain_of_salvation','riverside','beardfish','echolyn','eloy','john_wetton','medina_azahara','mostly_autumn','pendragon','rafo_r_ez','the_meteors','against_me','anti_flag','banda_bassotti','cadena_perpetua','descendents','distemper','dogwood','el_ltimo_ke_zierre','farben_lehre','toy_dolls','junkies','ksu','la_polla_records','la_vela_puerca','leatherface','less_than_jake','mad_caddies','millencolin','punkreas','reel_big_fish','snfu','stiff_little_fingers','swingin_utters','the_bouncing_souls','the_casualties','the_dickies','the_lawrence_arms','toyah','gerald_levert','gondwana','los_aut_nticos_decadentes','los_cafres','los_pericos','tryo','rakim_ken_y','billy_squier','bj_rn_afzelius','glay','hunters_collectors','john_entwistle','jokke','la_beriso','los_rancheros','los_tres','maanam','mikel_erentxun','peter_wolf','racoon','rev_lver','riblja_orba','sandro','gene_vincent','the_baseballs','stray_cats','as_marcianas','bruno_marrone','cristiano_ara_jo','fernando_sorocaba','joint_venture','serge_reggiani','ska_p','the_mighty_mighty_bosstones','fu_manchu','jay_jay_johanson','psyche','carlos_gardel'] {allow-input: true}
select_genre = "Classical" #@param ['Unknown','Acoustic','Alternative','Alternative Country','Alternative Dance','Alternative Hip Hop','Alternative Metal','Alternative Rock','American Folk','Americana','Art Punk','Art Rock','Avant Garde','Black Metal','Bluegrass','Blues','Blues Rock','Britpop','Celtic','Celtic Folk','Celtic Punk','Chanson','Children s Music','Classic Jazz','Classic Pop','Classical','Contemporary Folk','Contemporary Jazz','Country','Country Blues','Country Hip Hop','Country Jazz','Country Metal','Country Pop','Country Punk','Country Rock','Crossover','Crossover Thrash','Dance','Dance Pop','Dark Wave','Death Metal','Disco','Doom Metal','Dream Pop','Ebm','Electro','Electronic','Electronica','Electropop','Emo','Eurodance','Europop','Experimental','Experimental Hip Hop','Experimental Pop','Folk','Folk Metal','Folk Pop','Folk Punk','Folk Rock','French Pop','Funk','Gangsta Rap','Glam Metal','Gospel','Gothic','Gothic Metal','Gothic Rock','Grindcore','Grunge','Hard Rock','Hardcore Hip Hop','Hardcore Punk','Heavy Metal','Hip Hop','Hip Hop Rap','Horrorcore','House','Indie Folk','Indie Pop','Indie Rock','Industrial','Industrial Metal','Industrial Rock','J Pop','Jazz','Jazz Rock','K Pop','Latin','Latin Pop','Latin Rock','Lo Fi','Medieval','Melodic Death Metal','Melodic Hardcore','Metalcore','Movie','Mpb','Musical','Neofolk','New Wave','Nu Metal','Pop','Pop Punk','Pop Rock','Post Grunge','Post Hardcore','Post Punk','Power Metal','Progressive Death Metal','Progressive House','Progressive Jazz','Progressive Metal','Progressive Metalcore','Progressive Rock','Psychedelic Pop','Psychedelic Rock','Punk','Punk Rock','R N B','R N B Soul','Rap Metal','Rap Rock','Reggae','Rock','Rock N Roll','Rockabilly','Samba','Schlager','Sertanejo','Ska','Ska Punk','Soft Rock','Soul','Soul Pop','Soundtrack','Southern Hip Hop','Southern Rock','Swing','Symphonic Black Metal','Symphonic Metal','Symphonic Rock','Synthpop','Tejano','Television','Thrash Metal','Trance','Trip Hop','Vocal','Vocal Jazz','Vocal Pop','Worship'] {allow-input: true}
metas = [dict(artist = select_artist,
            genre = select_genre,
            total_length = hps.sample_length,
            offset = 0,
            lyrics = your_lyrics,
            ),
          ] * hps.n_samples
labels = [None, None, top_prior.labeller.get_batch_labels(metas, 'cuda')]

sampling_temperature = .96 #@param {type:"number"}

if gpu_info[0].find('Tesla T4') >= 0:
  lower_batch_size = 14
  print('Tesla T4 detected, lower_batch_size set to 12')
elif gpu_info[0].find('Tesla K80') >= 0:
  lower_batch_size = 8
  print('Tesla K80 detected, lower_batch_size set to 8')
elif gpu_info[0].find('Tesla P100') >= 0:
  lower_batch_size = 16
  print('Tesla P100 detected, lower_batch_size set to 16')
elif gpu_info[0].find('Tesla V100') >= 0:
  lower_batch_size = 16
  print('Tesla V100 detected, lower_batch_size set to 16')
elif gpu_info[0].find('A100') >= 0:
  lower_batch_size = 48
  print('Tesla A100 detected, lower_batch_size set to 48')
else:
  lower_batch_size = 8
  print('Different GPU detected, lower_batch_size set to 8.')
lower_level_chunk_size = 32
sampling_kwargs = [dict(temp=.99, fp16=True, max_batch_size=lower_batch_size,
                        chunk_size=lower_level_chunk_size),
                    dict(temp=0.99, fp16=True, max_batch_size=lower_batch_size,
                         chunk_size=lower_level_chunk_size),
                    dict(temp=sampling_temperature, fp16=True,
                         max_batch_size=max_batch_size, chunk_size=chunk_size)]

if sample_hps.mode == 'ancestral':
  zs = [t.zeros(hps.n_samples,0,dtype=t.long, device='cpu') for _ in range(len(priors))]
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
elif sample_hps.mode == 'upsample':
  assert sample_hps.codes_file is not None
  # Load codes.
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cpu() for z in data['zs']]
  assert zs[-1].shape[0] == hps.n_samples, f"Expected bs = {hps.n_samples}, got {zs[-1].shape[0]}"
  del data
  print('One click upsampling!')
elif sample_hps.mode == 'primed':
  assert sample_hps.audio_file is not None
  audio_files = sample_hps.audio_file.split(',')
  duration = (int(sample_hps.prompt_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
  x = load_prompts(audio_files, duration, hps)
  zs = top_prior.encode(x, start_level=0, end_level=len(priors), bs_chunks=x.shape[0])
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
elif sample_hps.mode == 'continue':
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cuda() for z in data['zs']]
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
else:
  raise ValueError(f'Unknown sample mode {sample_hps.mode}.')

# Set this False if you are on a local machine that has enough memory (this allows you to do the
# lyrics alignment visualization during the upsampling stage). For a hosted runtime,
# we'll need to go ahead and delete the top_prior if you are using the 5b_lyrics model.
if True:
  del top_prior
  empty_cache()
  top_prior=None
upsamplers = [make_prior(setup_hparams(prior, dict()), vqvae, 'cpu') for prior in priors[:-1]]
labels[:2] = [prior.labeller.get_batch_labels(metas, 'cuda') for prior in upsamplers]

zs = upsample(zs, labels, sampling_kwargs, [*upsamplers, top_prior], hps)

disconnect_runtime_after_finish = True #@param {type: "boolean"}
if disconnect_runtime_after_finish == True:
  from google.colab import runtime
  runtime.unassign()

# Guide to the above settings:

**your_lyrics:** Specify the lyrics Jukebox should attempt to follow. You can paste any lyrics you want in here or leave it blank, which will result in gibberish.

**model:**
OpenAI has trained a few different models for Jukebox. In this notebook, you can access the 5b_lyrics, 5b and 1b_lyrics models. Generally, the 1b_lyrics model doesn't produce as good results as the 5b ones.

(5b is like 5b_lyrics, but it doesn't support custom lyrics, so it will always generate gibberish vocals)

**hps.n_samples:**
Here you can choose how many samples you want to generate. Different GPUs can handle a different amount of samples. Recommended settings:

* T4 GPU: 2 samples
* P100 GPU: 3 samples
* A100 GPU: 6 samples

**hps.name:** Specifies the name of the folder in Google Drive, where you will find your results in. Make sure to choose a different name for each of your runs, or else the notebook will try to continue an older project of yours.

**speed_upsampling:** If selected, will upsample much faster, at the cost of the samples sounding slightly "choppy".

**mode:** Available modes are primed and ancestral. Primed will continue an already existing song, ancestral generates a song from scratch. (Upsample mode will be selected automatically if a data file is detected within the folder provided)

**audio_file:** Only needed for primed mode. Specifies which song Jukebox will continue. Upload the file you want (preferred .wav format, but mp3 is supported as well) to the root directory of your Google Drive and fill in its name above.

**prompt_length_in_seconds:** Only needed for primed mode. Specifies how many seconds of your file Jukebox will be primed on (so, at which point Jukebox  will "kick in"). Recommended to keep below 24 seconds for memory reasons.

**sample_length_in_seconds:** Specifies how long your fully generated samples are going to be.

**select_artist and select_genre:** List of available artists and genres can be found here: https://github.com/openai/jukebox/tree/master/jukebox/data/ids
The 5b_lyrics (and 5b) model utilizes the v2 lists, the 1b_lyrics model the v3 lists. It is possible to combine up to five v2 genres, for example "Hip Hop Pop Punk Disco". Combining v3 genres is not possible.

**sampling_temperature:** Determines the creativity and energy of Jukebox. The higher the temperature, the more chaotic and intense the result will be. You can experiment with this. Recommended to keep between .96 and .999

**disconnect_runtime_after_finish:** Automatically disconnects your session after your samples are fully generated and upsampled. Saves precious Colab credits.

Important links:

Official blog: https://openai.com/blog/jukebox/
Original repo: https://github.com/openai/jukebox/

License: Non-commercial, for details see: https://github.com/openai/jukebox/blob/master/LICENSE


Special thanks (in no particular order) goes out to:
* Michaels Lab
* Zags
* Jimney Baltigor
* Reedmayhew
* SMarioMan